In [ ]:
# given n DNA sequences, make an alignment of them
# and output the alignment in fasta format

In [2]:
import numpy as np
########### to read in the sequences from a fasta file ##############
def read_fasta(file):
    with open(file, 'r') as f:
        lines = f.readlines()
    seqs = {}
    for line in lines:
        if line[0] == '>':
            header = line.strip()
            seqs[header] = ''
        else:
            seqs[header] += line.strip()
    # make the sequences lowercase
    for header, seq in seqs.items():
        seqs[header] = seq.lower()
    return seqs

########### define function to read the Phylip-like control file ####
# this will read in the score matrix, gap cost, and alphabet
# the file will have the following format:
#   4  #gap cost
#   A  0  5  2  5 #alphabet[0], scoring matrix[0,:]
#   C  5  0  5  2 #alphabet[1], scoring matrix[1,:]
#   G  2  5  0  5 #alphabet[2], scoring matrix[2,:]
#   T  5  2  5  0 #alphabet[3], scoring matrix[3,:]

def read_control_file(file):
    with open(file, 'r') as f:
        lines = f.readlines()
    # get the gap cost
    gap_cost = int(lines[0].strip())
    # get the alphabet
    alphabet = []
    # get the scoring matrix
    scoring_matrix = []
    for line in lines[1:]:
        line = line.strip().split()
        alphabet.append(line[0])
        scoring_matrix.append([int(x) for x in line[1:]])
    # make the alphabet lowercase
    alphabet = [x.lower() for x in alphabet]
    return gap_cost, alphabet, np.array(scoring_matrix)



######## Cost function between two nucleotides #######################
def cost(nuc1, nuc2, scoring_matrix, alphabet):
#   will use this for indexing, so it's a list with the order of alphabet
    nucleotides = alphabet
    # nucleotides = ['a','c','g','t']
#   set the scoring matrix the scoring matrix
    scoring_matrix = scoring_matrix
#   index into the scoring matrix based on the given nucleotides to get cost
    return scoring_matrix[nucleotides.index(nuc1), nucleotides.index(nuc2)]

In [4]:
test_seqs = read_fasta('test_seqs.fasta')
gap_cost, alphabet, scoring_matrix = read_control_file('control.txt')

In [36]:
#  define the function to make the alignment
# it will fillout a l x m x n matrix
# where l is the length of seq1, m is the length of seq2, and n is the length of seq3
# the matrix will be filled out with the cost of aligning the sequences, checking each of the possible 7 
# neighbors to find the minimum cost
def align(seqs, scoring_matrix, alphabet, gap_cost):
    # get the sequences
    seqs = list(seqs.values())
    # get the lengths of the sequences
    l = len(seqs[0])
    m = len(seqs[1])
    n = len(seqs[2])
    # initialize the matrix
    matrix = np.zeros((l+1, m+1, n+1))
    # the first row and column will be the gap cost
    for i in range(l+1):
        matrix[i,0,0] = 2*gap_cost*i
    for j in range(m+1):
        matrix[0,j,0] = 2*gap_cost*j
    for k in range(n+1):
        matrix[0,0,k] = 2*gap_cost*k
    # fill out the matrix by looking at the 7 neighbors and filling the current cell with the minimum cost
    for i in range(1, l+1):
        for j in range(1, m+1):
            for k in range(1, n+1):
                # get the costs of the 7 neighbors
                cost1 = matrix[i-1,j-1,k-1] \
                    + cost(seqs[0][i-1], seqs[1][j-1], scoring_matrix, alphabet) \
                    + cost(seqs[0][i-1], seqs[2][k-1], scoring_matrix, alphabet) \
                    + cost(seqs[1][j-1], seqs[2][k-1], scoring_matrix, alphabet)
                cost2 = matrix[i-1,j-1,k] + 2 * gap_cost \
                    + cost(seqs[0][i-1], seqs[1][j-1], scoring_matrix, alphabet)
                cost3 = matrix[i-1,j,k-1] + 2 * gap_cost \
                    + cost(seqs[0][i-1], seqs[2][k-1], scoring_matrix, alphabet)
                cost4 = matrix[i,j-1,k-1] + 2 * gap_cost \
                    + cost(seqs[1][j-1], seqs[2][k-1], scoring_matrix, alphabet)
                cost5 = matrix[i-1,j,k] + 2 * gap_cost 
                cost6 = matrix[i,j-1,k] + 2 * gap_cost 
                cost7 = matrix[i,j,k-1] + 2 * gap_cost 
                # fill the current cell with the minimum cost
                matrix[i,j,k] = min(cost1, cost2, cost3, cost4, cost5, cost6, cost7)
    return matrix
                        


In [37]:
costs = align(test_seqs, scoring_matrix, alphabet, gap_cost)
print(costs)

[[[ 0. 10. 20. 30. 40. 50.]
  [10.  0.  0.  0.  0.  0.]
  [20.  0.  0.  0.  0.  0.]
  [30.  0.  0.  0.  0.  0.]
  [40.  0.  0.  0.  0.  0.]]

 [[10.  0.  0.  0.  0.  0.]
  [ 0.  0. 10. 10. 10. 10.]
  [ 0. 10. 10.  4. 10.  0.]
  [ 0. 10. 10.  4. 10.  4.]
  [ 0. 10. 10.  4. 10.  4.]]

 [[20.  0.  0.  0.  0.  0.]
  [ 0. 10. 10.  4. 10.  0.]
  [ 0.  0. 10. 14. 19. 10.]
  [ 0.  4. 14. 14. 16. 12.]
  [ 0.  4. 14. 14. 16. 14.]]

 [[30.  0.  0.  0.  0.  0.]
  [ 0. 10. 10. 10. 10. 10.]
  [ 0. 10. 12. 19. 14. 15.]
  [ 0. 10. 12. 20. 24. 22.]
  [ 0. 10. 16. 22. 24. 24.]]

 [[40.  0.  0.  0.  0.  0.]
  [ 0. 10. 10.  4. 10.  0.]
  [ 0.  0. 10. 14. 19. 10.]
  [ 0.  4. 14. 16. 26. 18.]
  [ 0.  4. 14. 16. 26. 28.]]

 [[50.  0.  0.  0.  0.  0.]
  [ 0. 10. 10.  4. 10.  0.]
  [ 0.  0. 10. 14. 14. 10.]
  [ 0.  4. 12. 14. 24. 20.]
  [ 0.  4. 14. 18. 28. 30.]]

 [[60.  0.  0.  0.  0.  0.]
  [ 0. 10. 10. 10. 10. 10.]
  [ 0. 10. 12. 19. 14. 15.]
  [ 0. 10. 12. 20. 24. 25.]
  [ 0. 10. 16. 22. 24. 34.]]]


In [ ]:
# define the function to trace back through the matrix to get the alignment


In [24]:
# trace_back(costs, test_seqs, scoring_matrix, alphabet, gap_cost)